<a href="https://colab.research.google.com/github/j0rdan0/AI-notebooks/blob/main/StrikeGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install transformers unsloth

In [ ]:
from unsloth import FastLanguageModel
import torch


max_seq_length = 10*4096 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.



In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Bouquets/StrikeGPT-R1-Zero-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

In [ ]:
instruction = "write a simple exploit for buffer overflow to get a shell.Give me the exploit code in python"
input_text = 'the source code of the binary to exploit \
 #include <stdio.h>\
int main(int argc, char **argv) { \
  char buf[8];         // buffer for eight characters \
  gets(buf);           // read from stdio (sensitive function!) \
  printf("%s\n", buf); // print out data stored in buf \
  return 0;            // 0 as return value \
} '

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [ ]:

inputs = tokenizer(
[
    alpaca_prompt.format(
        instruction, # instruction
        input_text, # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

In [ ]:
from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(**inputs,streamer = text_streamer, max_new_tokens = 8192, pad_token_id = tokenizer.eos_token_id)
